# Improve RAG Systems with User Feedback

In this example notebook, we showcase how to collect user feedback with the LastMile AI Tracing SDK and improve your RAG systems using this feedback via the RAG Debugger tool.


## Notebook Outline
* [Introduction](#intro)
* [Setup](#setup)
* [Part 1: Log User Feedback](#part1)
  * [Trace your code](#trace_code)
  * [Annotate trace with user feedback](#annotate)
* [Part 2: Analyze User Feedback](#part2)
  * [View user feedback results](#view)
* [More Resources](#more)



<a name="intro"></a>
# Introduction
**User Feedback** is the feedback responses obtained from end users. These responses are typically related to the quality of the output of your application.

RAG Debugger allows you to incorporate user feedback into your trace via the LastMile Tracing SDK, providing valuable insights into how users perceive the performance your RAG system.

User feedback can be captured in various forms:

* **Feedback Score:** Users can rate a specific span, such as a LLM-generated output or a retrieved document (e.g., 👍 / 👎 or on a scale 1-10)
* **Expected Value:** Users can provide the expected or correct output for a span, which is stored in the expected field alongside input and output. This is useful for capturing corrections.
* **User Comment:** Users can share additional context or qualitative feedback using a free-form text field.

<a name="setup"></a>
# Setup

To begin, we need to install lastmile-eval library. We are using ipywidgets for this example so installing that here too.

In [ ]:
!pip install lastmile-eval --upgrade
!pip install ipywidgets

Import all modules used in this tutorial.

In [2]:
from lastmile_eval.rag.debugger.tracing import get_lastmile_tracer
from lastmile_eval.rag.debugger.api import LastMileTracer
from lastmile_eval.rag.debugger.tracing.sdk import get_span_id, get_trace_id
from openai import OpenAI
from IPython.display import display, clear_output
from ipywidgets import Button, HBox, Output

Before we start this tutorial, we need the following tokens/keys:

* LastMile AI API Token: Go to the [LastMile Settings page](https://lastmileai.dev/settings?page=tokens). You will need to first create a LastMile AI account.
* OpenAI API Key: Go to [OpenAI API Keys page](https://platform.openai.com/account/api-keys) to create and access your OpenAI API Key.

You can either set these explicitly (uncomment the lines below), or save them in a `.env` file within this project directory.

In [3]:
import os

try:
    # First try this in case we're running on Google Collab
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] =  userdata.get('OPENAI_API_KEY')
    os.environ['LASTMILE_API_TOKEN'] =  userdata.get('LASTMILE_API_TOKEN')
except ModuleNotFoundError:
    import dotenv
    dotenv.load_dotenv()

# os.environ['OPENAI_API_KEY'] =  <OPENAI_API_KEY>
# os.environ['LASTMILE_API_TOKEN'] =  <LASTMILE_API_TOKEN>

<a name="part1"></a>

# Part 1: Log User Feedback
In this example, we generate a riddle using `gpt-3.5-turbo`. We trace our code with the LastMile AI Tracing SDK.

We collect user feedback on the quality of the riddle (👍/👎) and annotate this feedback to our trace.

<a name="trace_code"></a>

## Trace your code
First, instantiate a LastMile Tracer object.

The name of the `Tracer` is also your Project Name that will be visible in the RAG Debugger UI.

In [4]:
tracer: LastMileTracer = get_lastmile_tracer(
    tracer_name="tracer_0",
    project_name="User-Feedback",
)

The function generates our riddle using `gpt-3.5-turbo`.

In [9]:
# Decorate function with tracer
@tracer.trace_function("llm_call")
def generate()-> str:
    """
    Generates a riddle using OpenAI's GPT-3.5-turbo model.
    """
    user_query = "Tell me a riddle and give me the answer."
    system_prompt = "You must respond in capital letters."
    model = "gpt-3.5-turbo"

    response = OpenAI().chat.completions.create(
        messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_query}
        ],
        model = "gpt-3.5-turbo"
      )

    riddle = response.choices[0].message.content

    # Save span and trace IDs of generated riddle (key = riddle)
    # Use these IDs to link user feedback to a specific trace / span
    tracer.store_trace_id(key=riddle, span_id=get_span_id(), trace_id= get_trace_id())

    # Query Event - allows you to use Prompt Debugger in UI
    tracer.add_query_event(
          query=user_query,
          llm_output=riddle,
          system_prompt=system_prompt,
          metadata={"model": model}
      )

    return riddle

riddle = generate()

<a name="annotate"></a>

## Annotate trace with user feedback

The `get_and_store_user_feedback` function below generates a UI element to give a thumbs up/down.

We log the result (thumbs up/down reaction) back to the Trace and Span using the trace_id and span_id of our `generate` function.

In [ ]:
import trace


output = Output()

# Set trace_id and span_id so we can link user feedback to them
trace_id, span_id = tracer.read_trace_id(key=riddle)

# Generates a thumbs up/down UI for you to provide user feedback (try in Colab!)
def get_and_store_user_feedback():
    def on_thumbsup_clicked(b):
        with output:
            clear_output()
            print("You selected: 👍")
            tracer.log_binary_feedback(value=True, trace_id=trace_id)
            # Alternatively, you can log feedback using a string
            # tracer.log_feedback(span_id=span_id, trace_id=trace_id, feedback="👍")

    def on_thumbsdown_clicked(b):
        with output:
            clear_output()
            print("You selected: 👎")
            tracer.log_binary_feedback(value=False, trace_id=trace_id)
            # tracer.log_feedback(span_id=span_id, trace_id=trace_id, feedback="👎")

    thumbsup_button = Button(description="👍", button_style="success")
    thumbsup_button.on_click(on_thumbsup_clicked)

    thumbsdown_button = Button(description="👎", button_style="danger")
    thumbsdown_button.on_click(on_thumbsdown_clicked)

    display(HBox([thumbsup_button, thumbsdown_button]))
    display(output)

print(f'\n Generated Riddle:\n{riddle}\n Give it a thumbs up or thumbs down to provide feedback.\n')
get_and_store_user_feedback()

Now our user feedback has been logged to our Trace!

<a name="view"></a>
## View user feedback results
User feedback is 1:1 with a specific trace. You can view user feedback in the 'Traces' page of RAG Debugger.

#### From your terminal:

Export your LASTMILE_API_TOKEN

```bash
export LASTMILE_API_TOKEN="<your-api-token>"
```

Run this CLI command to access the UI

```bash
rag-debug launch
```
Go to the 'Traces' Page where you see all the Traces listed. One of the columns here is 'User Feedback'.
<img width="973" alt="Screenshot 2024-06-13 at 3 26 36 PM" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/6881194d-7fa3-42b2-9764-dfe4dafe2361">



Click on the Trace to get to the Trace Overview Page. You can then click on `llm-call' to access the prompt playground to iterate on the prompt.

<img width="973" alt="Screenshot 2024-06-13 at 3 32 01 PM" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/8e51ab6a-82f2-4a1f-8557-dee44124e68c">

Prompt Debugger

<img width="973" alt="Screenshot 2024-06-13 at 4 26 11 PM" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/cbd61565-b61a-4952-9a07-206cc43e7503">


<a name="more"></a>

# More Resources
* [RAG Debugger Documentation](https://rag.lastmileai.dev/docs/basics)
* [Discord - ask us anything!](https://discord.com/invite/xBhNKTetGx)